# Assignment 3 - Part 1

In [8]:
import urllib
import requests
import json
import math
import os

In [9]:
API = "http://gustav1.ux.uis.no:5002"

QUERY_FILE = "data/queries.txt"
QRELS_FILE = "data/qrels.csv"

TITLE_OUTPUT = "data/title_output.txt"
CONTENT_OUTPUT = "data/content_output.txt"
ANCHOR_OUTPUT = "data/anchors_output.txt"

INDEX = "clueweb12b"
ANCHOR_INDEX = "clueweb12b_anchors"



queries = {}
with open(QUERY_FILE, "r") as qfile:
    for line in qfile.readlines():
        qid, query = line.strip().split(" ", 1)
        queries[qid] = query
        
ground_truth = {}
with open(QRELS_FILE, "r") as gtruth:
    for line in gtruth.readlines():
        if line.startswith("QueryId"):
            continue
        # Relevance -> (-2, 0, 1, 2, 3)
        qid, doc_id, relevance = line.strip().split(",")
        if qid not in ground_truth:
            ground_truth[qid] = {}
        ground_truth[qid][doc_id] = int(relevance)
        

Issuing a search query againt the API

In [10]:

CACHE_DIR = "cache"
CACHE_DIR_SEARCH = CACHE_DIR + "/search"
CACHE_DIR_TERMVECTORS = CACHE_DIR + "/termvectors"

def search(indexname, query, field, size=10):
    cache_file = CACHE_DIR_SEARCH + "/" + indexname + "_" + query + "_" + field + "_" + str(size)
    url = "/".join([API, indexname, "_search"]) + "?" \
          + urllib.parse.urlencode({"q": query, "df": field, "size": size})
    if os.path.exists(cache_file):  # return from cache
        with open(cache_file) as infile:
            response = json.load(infile)
            return json.loads(response)
    else:
        with open(cache_file, "w") as outfile:
            response = requests.get(url).text
            json.dump(response, outfile)
            return json.loads(response)

Printing results for a given search query

In [11]:
CACHE_DIR_EXISTS = CACHE_DIR + "/exists"


# Check if document exists in index
def exists(doc_id):
    cache_file = CACHE_DIR_EXISTS + "/" + doc_id
    url = "/".join([API, INDEX, doc_id, "_exists"])
    if os.path.exists(cache_file):
        with open(cache_file) as infile:
            response = json.load(infile)
            doc = json.loads(response)
            
    else:
        with open(cache_file, "w") as outfile:
            response = requests.get(url).text
            json.dump(response, outfile)
            doc = json.loads(response)
    # Returns true if it exists, false if not
    print(doc)
    return doc.get("exists")

In [16]:

def get_baseline(fields):
    i = 1
    for field in fields:
        print(field)
        fout = "data/{}_output.txt".format(field)
        with open(fout, "w") as file:
            file.write("QueryId,DocumentId")
            for qid, query in queries.items():
                if field == "anchors":
                    res = search("clueweb12b_anchors", query, field, size=200)
                    for r in res.get("hits", {}).get("hits", {}):
                        print(i)
                        i += 1
                        if exists(r["_id"]):
                            file.write("\n{},{}".format(qid, r["_id"]))
                else: 
                    res = search(INDEX, query, field, size=20)
                    for r in res.get("hits", {}).get("hits", {}):
                        file.write("\n{},{}".format(qid, r["_id"]))
                    
def load_baseline(files):
    data = {}
    for field, file in files:
        data[field] = {}
        with open(file, "r") as f:
            for line in f.readlines():
                if line.startswith("QueryId"):
                    continue
                qid, docid = line.strip().split(",")
                if qid not in data[field]:
                    data[field][qid] = []
                data[field][qid].append(docid)
    return data
            

In [17]:
def ndcg(gains, ideal_gains, n):
    actual = gains[0]
    ideal = ideal_gains[0]
    for i in range(1, min(n, len(gains))):
        actual += gains[i] / math.log2(i + 1)
    for i in range(1, min(n, len(ideal_gains))):
        ideal += ideal_gains[i] / math.log2(i + 1)
    return actual / ideal

def evaluate(ranks, gtruth, field):
    ndcg10 = 0
    ndcg20 = 0
    for qid, ranking in ranks.items():
        gt = gtruth[qid]
        ideal_gains = sorted([relevance for _, relevance in gt.items()], reverse=True) 
        gains = []
        for doc_id in ranking:
            gains.append(gt.get(doc_id, 0)) if gt.get(doc_id, 0) >= 0 else gains.append(0)
        
        #Calculate ndcg
        ndcg10 += ndcg(gains, ideal_gains, 10)
        ndcg20 += ndcg(gains, ideal_gains, 20)
    print("Evaluating {}".format(field))
    print("NDCG@10")
    print(ndcg10 / len(ranks))
    print("NDCG@20")
    print(ndcg20 / len(ranks))
    print()

In [18]:
# Get baseline data
fields = ["content", "title", "anchors"]
get_baseline(fields)

content
title
anchors
1
{'exists': False}
2
{'exists': False}
3
{'exists': False}
4
{'exists': False}
5
{'exists': True}
6
{'exists': False}
7
{'exists': False}
8
{'exists': False}
9
{'exists': False}
10
{'exists': False}
11
{'exists': False}
12
{'exists': False}
13
{'exists': False}
14
{'exists': False}
15
{'exists': False}
16
{'exists': False}
17
{'exists': False}
18
{'exists': False}
19
{'exists': False}
20
{'exists': False}
21
{'exists': False}
22
{'exists': False}
23
{'exists': False}
24
{'exists': False}
25
{'exists': False}
26
{'exists': False}
27
{'exists': False}
28
{'exists': False}
29
{'exists': False}
30
{'exists': False}
31
{'exists': False}
32
{'exists': False}
33
{'exists': False}
34
{'exists': False}
35
{'exists': False}
36
{'exists': False}
37
{'exists': False}
38
{'exists': False}
39
{'exists': False}
40
{'exists': False}
41
{'exists': False}
42
{'exists': False}
43
{'exists': False}
44
{'exists': False}
45
{'exists': False}
46
{'exists': True}
47
{'exists': True}
48


796
{'exists': False}
797
{'exists': False}
798
{'exists': False}
799
{'exists': False}
800
{'exists': False}
801
{'exists': False}
802
{'exists': False}
803
{'exists': False}
804
{'exists': False}
805
{'exists': True}
806
{'exists': False}
807
{'exists': False}
808
{'exists': False}
809
{'exists': False}
810
{'exists': False}
811
{'exists': False}
812
{'exists': False}
813
{'exists': False}
814
{'exists': False}
815
{'exists': False}
816
{'exists': False}
817
{'exists': False}
818
{'exists': False}
819
{'exists': False}
820
{'exists': False}
821
{'exists': False}
822
{'exists': False}
823
{'exists': False}
824
{'exists': False}
825
{'exists': False}
826
{'exists': False}
827
{'exists': False}
828
{'exists': False}
829
{'exists': False}
830
{'exists': False}
831
{'exists': False}
832
{'exists': False}
833
{'exists': False}
834
{'exists': False}
835
{'exists': False}
836
{'exists': False}
837
{'exists': False}
838
{'exists': False}
839
{'exists': False}
840
{'exists': False}
841
{'exist

{'exists': False}
1301
{'exists': False}
1302
{'exists': False}
1303
{'exists': False}
1304
{'exists': False}
1305
{'exists': False}
1306
{'exists': False}
1307
{'exists': False}
1308
{'exists': False}
1309
{'exists': False}
1310
{'exists': False}
1311
{'exists': False}
1312
{'exists': False}
1313
{'exists': False}
1314
{'exists': False}
1315
{'exists': False}
1316
{'exists': False}
1317
{'exists': False}
1318
{'exists': False}
1319
{'exists': False}
1320
{'exists': False}
1321
{'exists': False}
1322
{'exists': False}
1323
{'exists': True}
1324
{'exists': False}
1325
{'exists': False}
1326
{'exists': False}
1327
{'exists': False}
1328
{'exists': False}
1329
{'exists': False}
1330
{'exists': True}
1331
{'exists': False}
1332
{'exists': False}
1333
{'exists': False}
1334
{'exists': False}
1335
{'exists': False}
1336
{'exists': False}
1337
{'exists': False}
1338
{'exists': False}
1339
{'exists': False}
1340
{'exists': True}
1341
{'exists': False}
1342
{'exists': False}
1343
{'exists': Fal

1897
{'exists': True}
1898
{'exists': True}
1899
{'exists': False}
1900
{'exists': True}
1901
{'exists': False}
1902
{'exists': False}
1903
{'exists': False}
1904
{'exists': False}
1905
{'exists': False}
1906
{'exists': False}
1907
{'exists': False}
1908
{'exists': False}
1909
{'exists': True}
1910
{'exists': False}
1911
{'exists': False}
1912
{'exists': False}
1913
{'exists': False}
1914
{'exists': False}
1915
{'exists': False}
1916
{'exists': False}
1917
{'exists': False}
1918
{'exists': False}
1919
{'exists': False}
1920
{'exists': False}
1921
{'exists': False}
1922
{'exists': False}
1923
{'exists': True}
1924
{'exists': False}
1925
{'exists': False}
1926
{'exists': False}
1927
{'exists': False}
1928
{'exists': False}
1929
{'exists': False}
1930
{'exists': False}
1931
{'exists': True}
1932
{'exists': False}
1933
{'exists': False}
1934
{'exists': True}
1935
{'exists': False}
1936
{'exists': False}
1937
{'exists': False}
1938
{'exists': False}
1939
{'exists': False}
1940
{'exists': Fa

{'exists': False}
2427
{'exists': False}
2428
{'exists': False}
2429
{'exists': False}
2430
{'exists': False}
2431
{'exists': True}
2432
{'exists': False}
2433
{'exists': False}
2434
{'exists': False}
2435
{'exists': False}
2436
{'exists': False}
2437
{'exists': False}
2438
{'exists': False}
2439
{'exists': False}
2440
{'exists': False}
2441
{'exists': False}
2442
{'exists': False}
2443
{'exists': False}
2444
{'exists': True}
2445
{'exists': False}
2446
{'exists': False}
2447
{'exists': False}
2448
{'exists': False}
2449
{'exists': False}
2450
{'exists': False}
2451
{'exists': False}
2452
{'exists': False}
2453
{'exists': False}
2454
{'exists': False}
2455
{'exists': False}
2456
{'exists': False}
2457
{'exists': False}
2458
{'exists': False}
2459
{'exists': False}
2460
{'exists': False}
2461
{'exists': False}
2462
{'exists': False}
2463
{'exists': False}
2464
{'exists': False}
2465
{'exists': False}
2466
{'exists': False}
2467
{'exists': False}
2468
{'exists': False}
2469
{'exists': Fa

...

6701
{'exists': False}
6702
{'exists': False}
6703
{'exists': True}
6704
{'exists': False}
6705
{'exists': False}
6706
{'exists': False}
6707
{'exists': False}
6708
{'exists': True}
6709
{'exists': False}
6710
{'exists': False}
6711
{'exists': False}
6712
{'exists': False}
6713
{'exists': False}
6714
{'exists': True}
6715
{'exists': False}
6716
{'exists': False}
6717
{'exists': False}
6718
{'exists': False}
6719
{'exists': False}
6720
{'exists': True}
6721
{'exists': False}
6722
{'exists': False}
6723
{'exists': False}
6724
{'exists': False}
6725
{'exists': False}
6726
{'exists': False}
6727
{'exists': False}
6728
{'exists': False}
6729
{'exists': False}
6730
{'exists': False}
6731
{'exists': True}
6732
{'exists': False}
6733
{'exists': False}
6734
{'exists': False}
6735
{'exists': False}
6736
{'exists': False}
6737
{'exists': False}
6738
{'exists': False}
6739
{'exists': False}
6740
{'exists': False}
6741
{'exists': True}
6742
{'exists': False}
6743
{'exists': True}
6744
{'exists': Fa

7346
{'exists': False}
7347
{'exists': False}
7348
{'exists': False}
7349
{'exists': False}
7350
{'exists': False}
7351
{'exists': False}
7352
{'exists': False}
7353
{'exists': False}
7354
{'exists': False}
7355
{'exists': False}
7356
{'exists': False}
7357
{'exists': False}
7358
{'exists': False}
7359
{'exists': False}
7360
{'exists': False}
7361
{'exists': False}
7362
{'exists': False}
7363
{'exists': False}
7364
{'exists': False}
7365
{'exists': False}
7366
{'exists': False}
7367
{'exists': False}
7368
{'exists': False}
7369
{'exists': False}
7370
{'exists': False}
7371
{'exists': False}
7372
{'exists': False}
7373
{'exists': False}
7374
{'exists': True}
7375
{'exists': True}
7376
{'exists': False}
7377
{'exists': False}
7378
{'exists': False}
7379
{'exists': False}
7380
{'exists': False}
7381
{'exists': False}
7382
{'exists': False}
7383
{'exists': False}
7384
{'exists': False}
7385
{'exists': False}
7386
{'exists': False}
7387
{'exists': False}
7388
{'exists': False}
7389
{'exists

8005
{'exists': False}
8006
{'exists': False}
8007
{'exists': False}
8008
{'exists': False}
8009
{'exists': False}
8010
{'exists': False}
8011
{'exists': False}
8012
{'exists': False}
8013
{'exists': False}
8014
{'exists': False}
8015
{'exists': False}
8016
{'exists': False}
8017
{'exists': False}
8018
{'exists': False}
8019
{'exists': False}
8020
{'exists': False}
8021
{'exists': False}
8022
{'exists': False}
8023
{'exists': False}
8024
{'exists': False}
8025
{'exists': False}
8026
{'exists': False}
8027
{'exists': False}
8028
{'exists': False}
8029
{'exists': False}
8030
{'exists': False}
8031
{'exists': False}
8032
{'exists': True}
8033
{'exists': False}
8034
{'exists': False}
8035
{'exists': False}
8036
{'exists': True}
8037
{'exists': False}
8038
{'exists': True}
8039
{'exists': False}
8040
{'exists': False}
8041
{'exists': False}
8042
{'exists': False}
8043
{'exists': False}
8044
{'exists': False}
8045
{'exists': False}
8046
{'exists': False}
8047
{'exists': False}
8048
{'exists'

{'exists': False}
8646
{'exists': False}
8647
{'exists': True}
8648
{'exists': False}
8649
{'exists': False}
8650
{'exists': False}
8651
{'exists': False}
8652
{'exists': False}
8653
{'exists': False}
8654
{'exists': False}
8655
{'exists': False}
8656
{'exists': False}
8657
{'exists': False}
8658
{'exists': False}
8659
{'exists': False}
8660
{'exists': False}
8661
{'exists': False}
8662
{'exists': False}
8663
{'exists': False}
8664
{'exists': False}
8665
{'exists': False}
8666
{'exists': False}
8667
{'exists': False}
8668
{'exists': False}
8669
{'exists': True}
8670
{'exists': False}
8671
{'exists': False}
8672
{'exists': False}
8673
{'exists': False}
8674
{'exists': False}
8675
{'exists': True}
8676
{'exists': False}
8677
{'exists': False}
8678
{'exists': False}
8679
{'exists': False}
8680
{'exists': False}
8681
{'exists': False}
8682
{'exists': False}
8683
{'exists': False}
8684
{'exists': True}
8685
{'exists': False}
8686
{'exists': False}
8687
{'exists': False}
8688
{'exists': Fals

9346
{'exists': False}
9347
{'exists': False}
9348
{'exists': False}
9349
{'exists': True}
9350
{'exists': False}
9351
{'exists': False}
9352
{'exists': False}
9353
{'exists': False}
9354
{'exists': False}
9355
{'exists': False}
9356
{'exists': False}
9357
{'exists': False}
9358
{'exists': False}
9359
{'exists': False}
9360
{'exists': False}
9361
{'exists': False}
9362
{'exists': True}
9363
{'exists': False}
9364
{'exists': False}
9365
{'exists': True}
9366
{'exists': False}
9367
{'exists': False}
9368
{'exists': False}
9369
{'exists': False}
9370
{'exists': False}
9371
{'exists': False}
9372
{'exists': True}
9373
{'exists': False}
9374
{'exists': False}
9375
{'exists': False}
9376
{'exists': False}
9377
{'exists': False}
9378
{'exists': False}
9379
{'exists': False}
9380
{'exists': False}
9381
{'exists': True}
9382
{'exists': False}
9383
{'exists': False}
9384
{'exists': False}
9385
{'exists': False}
9386
{'exists': False}
9387
{'exists': False}
9388
{'exists': False}
9389
{'exists': 

In [19]:
# Load data ...
files = [CONTENT_OUTPUT, TITLE_OUTPUT, ANCHOR_OUTPUT]
data = load_baseline(list(zip(fields, files)))

In [20]:
for field in fields:
    evaluate(data[field], ground_truth, field)

Evaluating content
NDCG@10
0.13799051375297441
NDCG@20
0.1282055279093094

Evaluating title
NDCG@10
0.12789484391213798
NDCG@20
0.11437855764306608

Evaluating anchors
NDCG@10
0.09019790802751133
NDCG@20
0.07799764406056864



## TODOs:

- Retrieve results for all queries (in `data/queries.txt`)
- Do it for both `title` and `content` fields (separately) and write the output to files
- Evaluate the results against the relevance judgments (in `data/qrels.csv`) in terms of NDCG@10 and NDCG@20